In [1]:
import os
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pickle
import numpy as np
import pandas as pd

# Cesta ke skriptům
sys.path.append(os.path.abspath("../scripts"))
from utils import Normalizer, create_multifeature_lstm_dataset

In [2]:
WINDOW_SIZE = 20
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5
PATIENCE = 5

results = []  # pro ukládání výsledků

In [3]:
with open("../data/preprocessed_datasets.pkl", "rb") as f:
    vsechny_datasety = pickle.load(f)

print(f"📊 Načteno {len(vsechny_datasety)} datasetů")

📊 Načteno 18 datasetů


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out)

In [5]:
for data in vsechny_datasety:
    ticker = data["ticker"]
    sektor = data["sector"]

    print(f"\n🚀 Trénuji model pro {ticker} ({sektor})")

    x_train = torch.tensor(data["x_train"], dtype=torch.float32)
    y_train = torch.tensor(data["y_train"], dtype=torch.float32)
    x_val = torch.tensor(data["x_val"], dtype=torch.float32)
    y_val = torch.tensor(data["y_val"], dtype=torch.float32)

    model = LSTMModel(input_size=x_train.shape[2])
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(TensorDataset(x_val, y_val), batch_size=BATCH_SIZE)

    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for xb, yb in train_loader:
            pred = model(xb).squeeze()
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                pred = model(xb).squeeze()
                loss = loss_fn(pred, yb)
                val_loss += loss.item()
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), f"../results/best_model_{ticker}.pth")
        else:
            patience_counter += 1

        if patience_counter >= PATIENCE:
            print(f"⏹️ Early stopping pro {ticker}")
            break

    results.append({
        "ticker": ticker,
        "sector": sektor,
        "train_loss": round(train_loss, 5),
        "val_loss": round(best_val_loss, 5)
    })


🚀 Trénuji model pro NVDA (Technology)


/opt/conda/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([29, 1])) that is different to the input size (torch.Size([29])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/conda/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, t

Epoch 1/50 - Train Loss: 0.0058 - Val Loss: 0.3971
Epoch 2/50 - Train Loss: 0.0057 - Val Loss: 0.3939
Epoch 3/50 - Train Loss: 0.0057 - Val Loss: 0.4044
Epoch 4/50 - Train Loss: 0.0056 - Val Loss: 0.3812
Epoch 5/50 - Train Loss: 0.0057 - Val Loss: 0.4118
Epoch 6/50 - Train Loss: 0.0057 - Val Loss: 0.4080
Epoch 7/50 - Train Loss: 0.0057 - Val Loss: 0.4027
Epoch 8/50 - Train Loss: 0.0057 - Val Loss: 0.4034
Epoch 9/50 - Train Loss: 0.0056 - Val Loss: 0.3971
⏹️ Early stopping pro NVDA

🚀 Trénuji model pro MSFT (Technology)
Epoch 1/50 - Train Loss: 0.0696 - Val Loss: 0.2209
Epoch 2/50 - Train Loss: 0.0424 - Val Loss: 0.2842
Epoch 3/50 - Train Loss: 0.0423 - Val Loss: 0.2523
Epoch 4/50 - Train Loss: 0.0421 - Val Loss: 0.2486
Epoch 5/50 - Train Loss: 0.0422 - Val Loss: 0.2658
Epoch 6/50 - Train Loss: 0.0424 - Val Loss: 0.2891
⏹️ Early stopping pro MSFT

🚀 Trénuji model pro AAPL (Technology)
Epoch 1/50 - Train Loss: 0.0678 - Val Loss: 0.1852
Epoch 2/50 - Train Loss: 0.0495 - Val Loss: 0.2129
E

In [6]:
results_df = pd.DataFrame(results)
results_df.to_csv("../results/all_model_results.csv", index=False)
print("✅ Výsledky uloženy do all_model_results.csv")
results_df.head()

✅ Výsledky uloženy do all_model_results.csv


,ticker,sector,train_loss,val_loss
0,NVDA,Technology,0.00564,0.38117
1,MSFT,Technology,0.04239,0.22091
2,AAPL,Technology,0.04896,0.16654
3,META,Communication,0.01064,0.20819
4,DIS,Communication,0.05884,0.03225
